In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler, scale
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score
import xgboost
from xgboost import XGBRegressor
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [2]:
X = pd.read_csv('./dataset-0510/train.csv', index_col='building_id')
X_test = pd.read_csv('./dataset-0510/test.csv', index_col='building_id')

y = X.total_price
X.drop(columns=['total_price'], inplace=True)

In [3]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.3, random_state=42) 

In [4]:
# step1. Imputation transformer for completing missing values.
step1 = ('Imputer', Imputer())
# step2. MinMaxScaler
step2 = ('MinMaxScaler', MinMaxScaler())
# step3. feature selection
#step3 = ('FeatureSelection', SelectFromModel(RandomForestRegressor()))
step3 = ('FeatureSelection', VarianceThreshold())

finally_step = ('model',
                XGBRegressor(
                    subsample=0.6,
                    reg_lambda=1e-5,
                    reg_alpha=0.1,
                    n_estimators=100000,
                    min_child_weight=6,
                    max_depth=9,
                    learning_rate=0.03,
                    gamma=0.03,
                    colsample_bytree=0.6,
                    tree_method='gpu_hist',
                    predictor='gpu_predictor',
                )
               )
parameters_for_testing = {
    'model__colsample_bytree':[0.4,0.6,0.8, 1],
    'model__gamma':[0, 0.1, 0.2, 0.3, 0.4],
    'model__min_child_weight':[1.5,6,10],
    'model__learning_rate':[0.0001, 0.001, 0.01, 0.1],
    'model__max_depth':[3, 6, 9],
    'model__n_estimators':[10000, 13000, 15000],
    'model__reg_alpha':[0, 1e-5, 1e-2,  0.01, 1],
    'model__reg_lambda':[0, 1e-5, 1e-2, 0.45, 1],
    'model__subsample':[0.6,0.95]
}

pipeline = Pipeline(steps=[step1, step2, step3, finally_step])

random_search = RandomizedSearchCV(pipeline,
                                  parameters_for_testing)



/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


## Tuning

### gamma
[0, 0.1, 0.2, 0.3, 0.4] -> 0

### subsample & colsample bytree
{'model__colsample_bytree': 0.7, 'model__subsample': 0.6}

### reg_alpha
best params
{'model__reg_alpha': 0.1}
best score
0.7540034509599658

In [5]:
random_search.fit(X_train, y_train)

NameError: name 'grid_search' is not defined

In [29]:
print('result')
print(grid_search.cv_results_)

print('best params')
print (grid_search.best_params_)

print('best score')
print (grid_search.best_score_)



result
{'mean_fit_time': array([6.07896821, 6.06961028, 6.05445385, 6.05004676, 6.05188918]), 'std_fit_time': array([0.30693766, 0.29511653, 0.29249228, 0.29546132, 0.29478944]), 'mean_score_time': array([0.13918686, 0.13923542, 0.13780228, 0.13799063, 0.14084395]), 'std_score_time': array([0.00029867, 0.0006985 , 0.00100802, 0.00103989, 0.00142107]), 'param_model__reg_alpha': masked_array(data=[0, 0.001, 0.005, 0.01, 0.05],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'model__reg_alpha': 0}, {'model__reg_alpha': 0.001}, {'model__reg_alpha': 0.005}, {'model__reg_alpha': 0.01}, {'model__reg_alpha': 0.05}], 'split0_test_score': array([0.76197589, 0.76197589, 0.76197589, 0.76197589, 0.76197589]), 'split1_test_score': array([0.77929524, 0.77929524, 0.77929524, 0.77929524, 0.77929524]), 'split2_test_score': array([0.7207392, 0.7207392, 0.7207392, 0.7207392, 0.7207392]), 'mean_test_score': array([0.75400344, 0.75400344, 

In [5]:
pipeline.fit(X, y)

[23:00:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('Imputer',
                 Imputer(axis=0, copy=True, missing_values='NaN',
                         strategy='mean', verbose=0)),
                ('MinMaxScaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('FeatureSelection', VarianceThreshold(threshold=0.0)),
                ('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=0.6, gamma=0.03,
                              importance_type='gain', learning_rate=0.03,
                              max_delta_step=0, max_depth=9, min_child_weight=6,
                              missing=None, n_estimators=100000, n_jobs=1,
                              nthread=None, objective='reg:linear',
                              predictor='gpu_predictor', random_state=0,
                              reg_alpha=0.1, reg_lambda=1e-05,
                 

## Train score

In [6]:
print(f"Train score: {pipeline.score(X_train, y_train)}")

Train score: 0.9999995034783626


In [7]:
y_pred = pipeline.predict(X_train)
y_pred

array([1.7996684e+06, 1.7426862e+07, 2.2284208e+06, ..., 3.1424387e+08,
       2.5341110e+06, 5.7789430e+06], dtype=float32)

In [8]:
#RMSLE = metrics.mean_squared_log_error(y_eval ,y_pred)
VARIANCE_SCORE = metrics.explained_variance_score(y_train, y_pred)
MSLE = np.sqrt(metrics.mean_squared_error(y_train, y_pred))
r2 = metrics.r2_score(y_train, y_pred)

print("MSLE = ", MSLE)
#print("RMSLE = ", RMSLE)
print("VARIANCE_SCORE = ", VARIANCE_SCORE)
print("R2", r2)
'''
0.03 lr
5w n_estimators
5080.7856385352
Train score: 0.9999994416836466
MSLE =  46193.834159808786
VARIANCE_SCORE =  0.9999992789294592
R2 0.9999992789294392
'''

'''
subsample=0.6,
reg_lambda=1e-5,
reg_alpha=0.1,
n_estimators=100000,
min_child_weight=6,
max_depth=9,
learning_rate=0.07,
gamma=0.03,
colsample_bytree=0.6,
tree_method='gpu_hist',
predictor='gpu_predictor',
    
train X_train 4500
MSLE =  12502.859103996803
VARIANCE_SCORE =  0.9999999471764185
R2 0.9999999471763487

train X 4800
MSLE =  36947.36986818785
VARIANCE_SCORE =  0.999999538707256
R2 0.9999995387070688
'''

MSLE =  38332.241695028824
VARIANCE_SCORE =  0.9999995034783824
R2 0.9999995034783626


"\nsubsample=0.6,\nreg_lambda=1e-5,\nreg_alpha=0.1,\nn_estimators=100000,\nmin_child_weight=6,\nmax_depth=9,\nlearning_rate=0.07,\ngamma=0.03,\ncolsample_bytree=0.6,\ntree_method='gpu_hist',\npredictor='gpu_predictor',\n    \ntrain X_train 4500\nMSLE =  12502.859103996803\nVARIANCE_SCORE =  0.9999999471764185\nR2 0.9999999471763487\n\ntrain X 4800\nMSLE =  36947.36986818785\nVARIANCE_SCORE =  0.999999538707256\nR2 0.9999995387070688\n"

## Test score

In [12]:
print(f"Test score: {pipeline.score(X_eval, y_eval)}")

Test score: 0.9999999587506768


In [13]:
y_pred = pipeline.predict(X_eval)
y_pred

array([ 7002881. ,  3899146.8,  9028630. , ...,  4313575. , 51753584. ,
       19441034. ], dtype=float32)

In [14]:
#RMSLE = metrics.mean_squared_log_error(y_eval ,y_pred)
VARIANCE_SCORE = metrics.explained_variance_score(y_eval, y_pred)
MSLE = np.sqrt(metrics.mean_squared_error(y_eval, y_pred))
r2 = metrics.r2_score(y_eval, y_pred)

print("MSLE = ", MSLE)
#print("RMSLE = ", RMSLE)
print("VARIANCE_SCORE = ", VARIANCE_SCORE)
print("R2", r2)

'''
0.03 lr
5080.7856385352
Test score: 0.9999999077079832
MSLE =  27300.38882949357
VARIANCE_SCORE =  0.9999997714168822
R2 0.9999997714128004
'''

'''
subsample=0.6,
reg_lambda=1e-5,
reg_alpha=0.1,
n_estimators=100000,
min_child_weight=6,
max_depth=9,
learning_rate=0.07,
gamma=0.0,
colsample_bytree=0.7,
tree_method='gpu_hist',
predictor='gpu_predictor',

train X_train 4500
MSLE =  6636.354426968303
VARIANCE_SCORE =  0.9999999864928084
R2 0.9999999864925494

train X 4800
MSLE =  6426.133544374753
VARIANCE_SCORE =  0.9999999873348979
R2 0.9999999873347509
'''

MSLE =  11597.15047626119
VARIANCE_SCORE =  0.9999999587509055
R2 0.9999999587506768


"\nsubsample=0.6,\nreg_lambda=1e-5,\nreg_alpha=0.1,\nn_estimators=100000,\nmin_child_weight=6,\nmax_depth=9,\nlearning_rate=0.07,\ngamma=0.0,\ncolsample_bytree=0.7,\ntree_method='gpu_hist',\npredictor='gpu_predictor',\n\ntrain X_train 4500\nMSLE =  6636.354426968303\nVARIANCE_SCORE =  0.9999999864928084\nR2 0.9999999864925494\n\ntrain X 4800\nMSLE =  6426.133544374753\nVARIANCE_SCORE =  0.9999999873348979\nR2 0.9999999873347509\n"

# Validation

In [17]:
scores = cross_val_score(pipeline, X, y, cv=5)


/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


[23:25:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


[23:40:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


[23:54:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


[00:10:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/islab/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


[00:25:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [18]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.77 (+/- 0.13)


In [19]:
# save test predictions to file
predictions = pipeline.predict(X_test)
output = pd.DataFrame({'building_id': X_test.index, 'total_price': predictions})
output.to_csv('submission/XGB_predict.csv', index=False)

In [20]:
output

,building_id,total_price
0,X5gsdTWGS3W7JJQB,1.754993e+07
1,BTshNOJyKHnT2YIT,3.980313e+06
2,dhdymr0lV8N5kZOT,9.931222e+06
3,VEwyGGMcD56w5BOc,3.085157e+06
4,wmUeMoJZfsqaSX9b,1.355154e+06
5,EtBjGAHmHCe9t7TZ,3.057929e+06
6,hPNH34vmaZtvBtqc,1.215509e+07
7,wXjeI38bYDMJJwZC,6.222240e+06
8,fxZSGX6aPAFKU8W4,1.665269e+06
9,ewr0Fx6ign87OwaV,4.407848e+06
